In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("asianpopsouthbay.csv")
df.dtypes
df["Zip Code"]= df["Zip Code"].astype(str) 
df.dtypes
df.shape

In [ ]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

In [ ]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [ ]:
address = 'San Jose, California '

geolocator = Nominatim(user_agent="sanjose")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Jose are {}, {}.'.format(latitude, longitude))

In [ ]:
cali_geo = r'ca_california_zip_codes_geo.min.json'

In [ ]:
map_sanjose = folium.Map(location=[latitude, longitude], zoom_start=10)
map_sanjose.choropleth(
    geo_data=cali_geo,
    data=df,
    columns=['Zip Code', '% Asians'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='% Asians'
)

map_sanjose

In [ ]:
def getNearbyVenues(zipcode, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for zipcode, lat, lng in zip(zipcode, latitudes, longitudes):
        print(zipcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            zipcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
sanjose_venues = getNearbyVenues(zipcode=df['Zip Code'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [ ]:
# one hot encoding
sanjose_onehot = pd.get_dummies(sanjose_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sanjose_onehot['Zip Code'] = sanjose_venues['Zip Code'] 

# move neighborhood column to the first column
fixed_columns = [sanjose_onehot.columns[-1]] + list(sanjose_onehot.columns[:-1])
sanjose_onehot = sanjose_onehot[fixed_columns]

sanjose_onehot.head()

In [ ]:
sanjose_grouped = sanjose_onehot.groupby('Zip Code').mean().reset_index()
sanjose_grouped.head(50)

In [ ]:
num_top_venues = 5

for hood in sanjose_grouped['Zip Code']:
    print("----"+hood+"----")
    temp = sanjose_grouped[sanjose_grouped['Zip Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Zip Code'] = sanjose_grouped['Zip Code']

for ind in np.arange(sanjose_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sanjose_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(50)

In [ ]:
# set number of clusters
kclusters = 5

sanjose_grouped_clustering = sanjose_grouped.drop('Zip Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sanjose_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sanjose_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sanjose_merged = sanjose_merged.join(neighborhoods_venues_sorted.set_index('Zip Code'), on='Zip Code')

sanjose_merged.head(100) # check the last columns!


In [ ]:
sanjose_merged.sort_values(['% Asians'], ascending=False)

In [ ]:
sanjose_merged1=sanjose_merged.dropna()

In [ ]:
sanjose_merged2=sanjose_merged1.join(sanjose_grouped.set_index('Zip Code'), on='Zip Code')

In [ ]:
sanjose_merged3 = sanjose_merged2.groupby('Cluster Labels', as_index=False).sum()

In [ ]:
sanjose_merged3.to_csv('out4.csv')
sanjose_merged4 = sanjose_merged3.drop(['Longitude','Latitude','% Asians'], axis=1)
sanjose_merged4.head()

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cluster Labels']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Cluster Labels'] = sanjose_merged4['Cluster Labels']

for ind in np.arange(sanjose_merged3.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(sanjose_merged4.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted1.head()

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sanjose_merged1['Latitude'], sanjose_merged1['Longitude'], sanjose_merged1['Zip Code'], sanjose_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster1 = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster1-1],
        fill=True,
        fill_color=rainbow[cluster1-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters.choropleth(
    geo_data=cali_geo,
    data=df,
    columns=['Zip Code', '% Asians'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='% Asians'
)

map_clusters